In [1]:
!pip install web3

In [2]:
import time
import json
import requests
import pandas as pd
from tqdm import tqdm
from web3 import Web3
from web3.exceptions import ABIFunctionNotFound

In [3]:
POLY_API_KEY = "TYVT9VBDKZUHWS35GYYJP3FK7MTS8XS8K9"
POLY_URL = "https://api.polygonscan.com/api"

web3ProviderURL = "https://rpc.ankr.com/polygon/b1bdd2819d3da7be4760d270af7ef985a74725cbe337c348bca633d096806c5a"
w3 = Web3(Web3.HTTPProvider(web3ProviderURL))

getAccountBalance_url = "https://rpc.ankr.com/multichain/79258ce7f7ee046decc3b5292a24eb4bf7c910d7e39b691384c7ce0cfb839a01/?ankr_getAccountBalance="

In [4]:
factories = {
    "quickswap": {
        "address" : ["0x5757371414417b8c6caad45baef941abc7d3ab32"],
        "pool" : []
        }
}

In [5]:
def get_abi(address):
    params = {
        "module": "contract",
        "action": "getabi",
        "address": address,
        "apikey": POLY_API_KEY
    }
    response = requests.post(POLY_URL, params=params)
    time.sleep(0.2)
    data = json.loads(response.text)
    result = data['result']
    return result

In [6]:
def get_contract(address):
    abi = get_abi(address)
    checksum_address = Web3.to_checksum_address(address)
    contract = w3.eth.contract(address=checksum_address, abi=abi)
    return contract

In [7]:
def get_fee_from_fee(contract, name):
    deno_dict = {
        "uniswap_v3" : 10**6,
        "sushiswap_v3" : 10**6,
        "curve" : 10**10,
    }
    if name in deno_dict:
        fee = float(contract.functions.fee().call())
        fee = fee / deno_dict[name]
        return fee
    return None


def get_fee_from_swapFeeUnits(contract, name):
    if name == "kyberswap":
        swapFeeUnits = float(contract.functions.swapFeeUnits().call())
        fee = swapFeeUnits / 100000
        return fee
    return None


def get_fee_from_globalState(contract, name):
    if name == "quickswap_v3":
        globalState = contract.functions.globalState().call()
        fee = globalState[2]
        fee = fee / 1000
        return fee
    return None


def get_fee(contract, name):
    fix_fee_dict = {
        "quickswap": 3/1000,
        "sushiswap" : 3/1000,
        "apeswap" : 2/1000,
        "dfyn" : 3/1000,
        "jetswap" : 1/1000,
    }

    fee_fun = [
        get_fee_from_fee,
        get_fee_from_swapFeeUnits,
        get_fee_from_globalState,
    ]

    if name in fix_fee_dict:
        return fix_fee_dict[name]

    for fun in fee_fun:
        fee = fun(contract, name)
        if fee:
            return fee
    return fee

In [8]:
def get_tokens(contract):
    try:
        token_1 = contract.functions.token0().call().lower()
        token_2 = contract.functions.token1().call().lower()
        return token_1, token_2
    except Exception:
        return None, None

In [9]:
def get_pool_from_allPairs(factory, start, end):
    rows = []
    name = factory
    for factory_address in factories[factory]["address"]:
        factory_contract = get_contract(factory_address)
        try:
            allPairsLength = factory_contract.functions.allPairsLength().call()
            if end > allPairsLength:
                end = allPairsLength
            for i in tqdm(range(start, end)):
                try:
                    pool_address = factory_contract.functions.allPairs(i).call().lower()
                    contract = get_contract(pool_address)
                    fee = get_fee(contract, name)
                    token_1, token_2 = get_tokens(contract)
                    row = {
                        "name" : name,
                        "factory" : factory_address,
                        "fee" : fee,
                        "contract" : pool_address,
                        "token_1" : token_1,
                        "token_2" : token_2,
                    }
                    rows.append(row)
                    time.sleep(0.2)
                except ValueError:
                    continue
        except ABIFunctionNotFound:
            continue
    pool = pd.DataFrame(rows)
    pool = pool.drop_duplicates(ignore_index = True)
    return pool

In [10]:
def get_tvl(pool):
    token_type = ['NATIVE']
    rows = []

    for i in tqdm(range(0, len(pool))):
        payload = {
            "jsonrpc": "2.0",
            "method": "ankr_getAccountBalance",
            "params": {
                "blockchain": ["polygon"],
                "walletAddress": pool.contract[i],
                "nativeFirst": True,
                "onlyWhitelisted": True,
                "pageSize": 2
            },
            "id": 1
        }
        headers = {
            "accept": "application/json",
            "content-type": "application/json"
        }

        response = requests.post(getAccountBalance_url, json=payload, headers=headers)
        time.sleep(1)
        if response.status_code == 200:
            try:
                data = json.loads(response.text)
                tvl = data['result']['totalBalanceUsd']
            except KeyError:
                tvl = None
            row = {
                "name" : pool.name[i],
                "factory" : pool.factory[i],
                "tvl" : tvl,
                "fee" : pool.fee[i],
                "contract" : pool.contract[i],
                "token_1" : pool.token_1[i],
                "token_2" : pool.token_2[i],
            }

            rows.append(row)
    pool = pd.DataFrame(rows)
    pool = pool.dropna(axis=0).reset_index(drop=True)
    return pool

In [11]:
for count in range(0, 10000):
    start = count * 1000
    end = start + 1000
    pool = get_pool_from_allPairs("quickswap", start, end)
    if len(pool) <= 0:
        break
    pool = get_tvl(pool)
    file_name = f"part{count}.csv"
    pool.to_csv(file_name, index=False)

100%|█████████████████████████████████████████| 413/413 [11:35<00:00,  1.68s/it]
0it [00:00, ?it/s]


In [19]:
files = [f"part{i}.csv" for i in range(count)]

In [20]:
combined_df = pd.DataFrame()

In [21]:
for file in files:
    df = pd.read_csv(file)
    combined_df = pd.concat([combined_df, df], ignore_index=True)

combined_df = combined_df.drop_duplicates()
combined_df

,name,factory,tvl,fee,contract,token_1,token_2
0,quickswap,0x5757371414417b8c6caad45baef941abc7d3ab32,4.427543e+02,0.003,0xd899e25f21222abe4e6212a7adb26190b5976268,0x55bee1bd3eb9986f6d2d963278de09ee92a3ef1d,0xf6ad3ccf71abb3e12becf6b3d2a74c963859adcd
1,quickswap,0x5757371414417b8c6caad45baef941abc7d3ab32,4.429294e+02,0.003,0xbb60d1f9cbaefa6845bf635db23ea25a27a11e83,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,0x55bee1bd3eb9986f6d2d963278de09ee92a3ef1d
2,quickswap,0x5757371414417b8c6caad45baef941abc7d3ab32,4.430230e+02,0.003,0xa3d74f0b4726019ecde5241e6b8b016a02eae1f2,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,0xf6ad3ccf71abb3e12becf6b3d2a74c963859adcd
3,quickswap,0x5757371414417b8c6caad45baef941abc7d3ab32,2.058289e+06,0.003,0x853ee4b2a13f8a742d64c8f088be7ba2131f670d,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,0x7ceb23fd6bc0add59e62ac25578270cff1b9f619
4,quickswap,0x5757371414417b8c6caad45baef941abc7d3ab32,8.486322e+04,0.003,0x4a35582a710e1f4b2030a3f826da20bfb6703c09,0x7ceb23fd6bc0add59e62ac25578270cff1b9f619,0x8f3cf7ad23cd3cadbd9735aff958023239c6a063
...,...,...,...,...,...,...,...
49393,quickswap,0x5757371414417b8c6caad45baef941abc7d3ab32,1.175453e+00,0.003,0x9a959f1ea80e922ed82cf17ba3518284af29d8c0,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,0x869db01a622a375240923bcbd983ffd454c48f0d
49394,quickswap,0x5757371414417b8c6caad45baef941abc7d3ab32,0.000000e+00,0.003,0x907d94acd1d763d63f831424f16899bc032718d9,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,0xd344e674887b853f9964d3efe69e2b31b0a1a812
49395,quickswap,0x5757371414417b8c6caad45baef941abc7d3ab32,0.000000e+00,0.003,0xebac6b261f71f36cae76d865daf88d0713ea2233,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,0x27c814ef0034977e1012467b66510a7f83797743
49396,quickswap,0x5757371414417b8c6caad45baef941abc7d3ab32,4.868098e+00,0.003,0x7d94b04cef1ca0088cedecb7dd59aecd27f3a82d,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,0x61162497debaad24cc027f804b690448126c3ab1


In [22]:
combined_df.to_csv('quickswap_combined.csv', index=False)